# Initialize

In [1]:
!pip install -U spacy==2.3.5

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-md 3.0.0 requires spacy<3.1.0,>=3.0.0, but you have spacy 2.3.5 which is incompatible.
You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.
  Using cached catalogue-1.0.0-py2.py3-none-any.whl (7.7 kB)
  Using cached srsly-1.0.5-cp39-cp39-win_amd64.whl (177 kB)
  Attempting uninstall: srsly
    Found existing installation: srsly 2.4.0
    Uninstalling srsly-2.4.0:
      Successfully uninstalled srsly-2.4.0
  Attempting uninstall: catalogue
    Found existing installation: catalogue 2.0.1
    Uninstalling catalogue-2.0.1:
      Successfully uninstalled catalogue-2.0.1


In [2]:
import spacy
print(spacy.__version__)

2.3.5


In [3]:
# Had to run this to make something below work.
# This is a donwgrade of numpy to a version without a bug.
# This is a temp solution until a new version of numpy has solved the bug.
!pip install numpy==1.19.3 --user

You should consider upgrading via the 'c:\users\lhustvei\appdata\local\programs\python\python39\python.exe -m pip install --upgrade pip' command.


In [4]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import get_data_from_text_service_item_url

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

url_start = "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com"
url_values_1 = "/regulation/2013/11/22/1404/chapter/3/section/24/part/2/subpart/1"
url_1 = url_start + url_values_1

#get_data_from_text_service_item_url(url_1)
get_data_from_text_service_item_url(url_values_1)

{'regulation_year': '2013',
 'regulation_month': '11',
 'regulation_day': '22',
 'regulation_id': '1404',
 'chapter_number': '3',
 'section_number': '24',
 'part_number': '2',
 'sub_part_number': '1'}

# Loop through text service chapter input

In [5]:
text_service_chapter_input = {
    "sections": [
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/7/ledd/1",
                    "content": "(1) Dette kapittelet gjelder for fartøy bygget etter 2. januar 1988.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "32291653-5b00-4af5-b0f4-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/7/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/7/ledd/2",
                    "content": "(2) Fartøy som er bygget før 2. januar 1988 skal være bygget og utrustet etter reglene som gjaldt da fartøyet ble bygget, med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "49110ac1-6879-40f5-b0f5-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/7/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/7/ledd/3",
                    "content": "(3) Når et fartøy forandres eller ombygges i vesentlig omfang, skal fartøyet oppfylle kravene til nytt fartøy.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "4d8b60fc-e93a-445d-b0f6-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/7/part/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/7",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p7",
            "title": "Kapittelets anvendelsesområde",
            "uid": "012d9722-dde2-4917-9e91-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/7"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/8/ledd/1",
                    "content": "(1) Fartøy bygget etter 1. januar 1992 skal brannsikres konstruksjonsmessig i henhold til kapittel Y14 i Nordisk Båt Standard (1990), med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "b3613ec8-5ba8-4d3f-b0f7-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/8/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/8/ledd/2",
                    "content": "(2) Fartøy som er bygget før 1. januar 1992 skal brannsikres konstruksjonsmessig etter reglene som gjaldt da fartøyet ble bygget.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "a6fda95b-b0a1-4cc2-b0f8-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/8/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/8/ledd/3",
                    "content": "(3) Uavhengig av fartøyets byggedato skal gassfyrte anlegg skal tilfredsstille forskrift 20. oktober 1983 nr. 1580 om sikringstiltak for gassfyrte anlegg m.m. som bruker propan eller andre lette kullvannstoffer anvendt ombord på fartøy.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "0e006bc1-a2fa-40d5-b0f9-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/8/part/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/8",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p8",
            "title": "Brannsikring",
            "uid": "dcdc807a-70c0-48de-9e92-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/8"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/1",
                    "content": "(1) Ovner med elektrisk krafttilførsel, skal ha en oljetilførsel som stenges automatisk hvis\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/9/ledd/1/liste/a",
                            "content": "ovnen overopphetes (maksimum 400 °C i røykgass),",
                            "uid": "93eb2b80-088d-4a6e-f392-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/1/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/9/ledd/1/liste/b",
                            "content": "flammen slukker,",
                            "uid": "fb607749-bd20-4c24-f393-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/1/subpart/2"
                        },
                        {
                            "number": "c)",
                            "referenceId": "/kapittel/3/paragraf/9/ledd/1/liste/c",
                            "content": "det oppstår svikt i lufttilførsel eller kraftforsyning.",
                            "uid": "02f6937a-f1e5-442f-f394-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/1/subpart/3"
                        }
                    ],
                    "sentences": [],
                    "uid": "90c03481-da21-42c0-b0fa-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/2",
                    "content": "(2) Ovner uten elektrisk krafttilførsel, skal ha en oljetilførsel som stenges automatisk hvis\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/9/ledd/2/liste/a",
                            "content": "ovnen overopphetes (maksimum 400 °C i røykgass),",
                            "uid": "059c57ac-48a6-44d7-f395-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/2/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/9/ledd/2/liste/b",
                            "content": "flammen slukker.",
                            "uid": "503c97fa-87df-4fac-f396-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/2/subpart/2"
                        }
                    ],
                    "sentences": [],
                    "uid": "cd628483-98ce-49e6-b0fb-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/3",
                    "content": "(3) Ovner med dryppregulering (stillkran med seglass), trenger ikke automatisk stengning av oljetilførsel når flammen slukker ved at oljeforbruket er begrenset av en oljestillerventil. Maksimalt tillatt høydeforskjell mellom oljetank og stillkran skal fremgå av installasjonsbeskrivelsen. Oljetrykksreduksjonsventil skal monteres når en tilfredsstillende oljeregulering ikke oppnås på grunn av trykkvariasjoner. Overflomssystemet skal kunne motta hele tankens volum.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "7c83ec10-abf3-41cb-b0fc-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/4",
                    "content": "(4) Tilbakestillingen av sikkerhetsfunksjonen skal være manuell.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ca77d44f-e360-40de-b0fd-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/4"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/5",
                    "content": "(5) Installasjonsbeskrivelse skal oppbevares om bord. Bruksanvisningen skal være oppslått i nærheten av ovnen.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "4bb3ad1c-cb37-4268-b0fe-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/5"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/6",
                    "content": "(6) Ovn med tilbehør skal installeres i overensstemmelse med installasjonsbeskrivelse fra produsent.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "e45dcee9-58cb-4b2f-b0ff-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/6"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/9/ledd/7",
                    "content": "(7) Ovn skal ha avtrekk til skorstein.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "65a599f8-a7ba-42f1-b100-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/9/part/7"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/9",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p9",
            "title": "Oljefyrt ovn",
            "uid": "a17a86bd-8884-41e6-9e93-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/9"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/1",
                    "content": "(1) Ovn skal være festet forsvarlig til dørk eller fast benk, om mulig nær fartøyets senterlinje.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ee145778-a6f4-493b-b101-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/2",
                    "content": "(2) Ovn uten automatisk stengning av oljetilførsel ved slukket flamme, skal ha egen oljetank på maksimalt 40 liter. Oljetank skal ikke fylles automatisk og skal plasseres i den høyde som er angitt i installasjonsbeskrivelsen.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "9329ecb9-4bf8-4a10-b102-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/3",
                    "content": "(3) Oljetank skal ha vannutskiller og plugg for avtapping av eventuelt vann, og være plassert utenfor det rom hvor ovnen er installert.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "d04ebd76-a3bb-4bd7-b103-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/4",
                    "content": "(4) Oljeledningen til regulator skal ha filter for utskilling av slam og vann. Filteret skal ikke plasseres over ovnen.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ebb7e8ef-383a-4a7d-b104-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/4"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/5",
                    "content": "(5) Det skal brukes rør av kopper eller likeverdig materiale mellom tank og ovn. Rørene skal festes og legges slik at luftlommer unngås.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "50107db8-1c7a-4aec-b105-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/5"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/6",
                    "content": "(6) Ovn skal ha overflomsåpninger med avløpsrør og oppsamlingsbeholder. Både ovn og eventuell flottørventil skal være tilknyttet oppsamlingsbeholderen.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "43fad3fe-114f-4c50-b106-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/6"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/7",
                    "content": "(7) Røropptak skal ikke ha mindre diameter enn ovnens avløpsstuss, og røret skal legges med minst mulig bend.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "4d41137c-fadd-4c27-b107-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/7"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/8",
                    "content": "(8) Drenering av kondensvann/slam, rensing av filtre, ettersyn av brennere og andre komponenter skal gjøres i tidsintervallene som er beskrevet i bruksanvisningen.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "786e1678-3bb2-4c2f-b108-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/8"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/9",
                    "content": "(9) I rom med ovn skal det være god ventilasjon.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "82fb6bfb-c746-4b6e-b109-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/9"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/10/ledd/10",
                    "content": "(10) På fartøy med største lengde 8 til 15 meter skal ovnen kontrolleres hos godkjent foretak innen seks måneder etter at den er installert.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "8ee68ade-7ba5-4d9d-b10a-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/10/part/10"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/10",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p10",
            "title": "Installasjon mv. av oljefyrt ovn",
            "uid": "01e23e11-a823-400c-9e94-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/10"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/11/ledd/1",
                    "content": "(1) Skott skal isoleres med ubrennbart materiale i henhold til vedlegg 4.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "e193910e-6f00-4223-b10b-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/11/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/11/ledd/2",
                    "content": "(2) Tredørk eller brennbart dekksbelegg under ovn skal beskyttes med stål eller aluminiumsplate under hele ildstedet og 15 cm foran ilegget. Er ovnen plassert på benk eller liknende, skal benken beskyttes med ubrennbart materiale.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "d4be86a1-07ce-412a-b10c-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/11/part/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/11",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p11",
            "title": "Isolasjon av ildsteder mv.",
            "uid": "d8e70b07-ff48-41cc-9e95-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/11"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/12/ledd/1",
                    "content": "(1) Røykopptak og eksosrør skal bestå av minst 3 mm tykke stålplater og festes forsvarlig. Røykopptaksrør skal føres minst 1,3 m over dekk.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "3c54fad0-f76d-41a1-b10d-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/12/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/12/ledd/2",
                    "content": "(2) Stenullskåler med minst 6 cm tykkelse skal plasseres mellom røykopptak og brennbart materiale. Mellom tetningsplater av metall og brennbart materiale skal det isoleres med ubrennbart materiale. Alternativt kan et rør med minst 6 cm større diameter enn røykopptaksrøret monteres utenpå dette, slik at det blir vertikal luftstrømming mellom rørene. Mellom ytre rør og brennbart materiale isoleres med stenullskåler eller mineralullsnor med 3 cm tykkelse. Røret skal føres minst 1 m over dekk, og utløpet beskyttes med krage.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ee16ecbb-e411-4a03-b10e-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/12/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/12/ledd/3",
                    "content": "(3) Når røykopptak føres opp langs skott, skal skottet isoleres med ubrennbart materiale. Røykopptak skal ikke plasseres nærmere isolert skott enn 10 cm. Avstanden til isolert skott kan reduseres til 5 cm dersom røykopptak (rør) er isolert med stenullskåler eller mineralullsnor av minst 2 cm tykkelse.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "d4e1cad1-b3b5-45ff-b10f-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/12/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/12/ledd/4",
                    "content": "(4) Eksosmanifold som ikke er vannavkjølt, og deler av eksosrør som kan forårsake antennelse, skal isoleres slik at oljen ikke kan trenge gjennom. Der gassene er vannavkjølt, kan andre arrangementer godtas. Eksosrør skal ikke være nærmere brennbart materiale enn 2 cm.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "9334da01-4cf0-40c4-b110-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/12/part/4"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/12/ledd/5",
                    "content": "(5) Oljeledninger skal legges lengst mulig bort fra eksosrør og varme maskindeler.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "16c2a15a-aa37-444d-b111-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/12/part/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/12",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p12",
            "title": "Røykopptak og eksosrør",
            "uid": "c3c9e85d-e334-4ec1-9e96-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/12"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/13/ledd/1",
                    "content": "Med ubrennbart materiale menes et materiale som ved prøving og vurdering i henhold til MSC.61(67) (FTP-koden) tilfredsstiller kravene til klassifisering av materialer som ubrennbare. Isolasjonen skal om nødvendig, for å bevare den brannisolerende effekten, beslås med tynnplate av galvanisert stål eller aluminium.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "c8566e59-12cb-46e9-b112-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/13/part/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/13",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p13",
            "title": "Isolering med ubrennbart materiale",
            "uid": "caf62674-d132-4560-9e97-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/13"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/14/ledd/1",
                    "content": "(1) Maskinrom i trefartøy skal ha dørk av stål eller aluminiumsplate. Maskinrom skal isoleres med ubrennbart materiale eller ha et godkjent brannslokningsanlegg.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "92fbe7a1-2236-4bc7-b113-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/14/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/14/ledd/2",
                    "content": "(2) Casing av tre, bjelker og underside av dekk og skott i for- og akterkant skal isoleres med ubrennbart materiale i maskinrom uten garnering.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "3c48db8b-6c7c-48cd-b114-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/14/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/14/ledd/3",
                    "content": "(3) Treverk skal isoleres med ubrennbart materiale ned til underkant av dørk i maskinrom med garnering. Dersom den øvrige isolasjon slutter tett inntil og rundt brenselsoljetanken kan isolasjonen bak tanken sløyfes.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "950ff816-3347-4d94-b115-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/14/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/14/ledd/4",
                    "content": "(4) Brennbart materiale som utsettes for varme fra eksosmanifolder, eksosrør etc., i fartøy hvor motoren ikke er i et eget rom, skal isoleres med ubrennbart materiale. Brukes det motorkassedeksel, skal denne isoleres innvendig og beslås med galvanisert stål- eller aluminiumsplate.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "48b11523-826c-4e6b-b116-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/14/part/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/14",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p14",
            "title": "Isolasjon av maskinrom på trefartøy",
            "uid": "3eac50b4-9d1a-44ec-9e98-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/14"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/15/ledd/1",
                    "content": "(1) Fartøy med største lengde mellom 10,67 og 15 meter skal ha brannslokningsanlegg. Når det er personer om bord skal brannslukkingsanlegget bare kunne utløses manuelt.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "881661ee-6bc4-467f-b117-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/15/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/15/ledd/2",
                    "content": "(2) Slokningsmiddel skal være av godkjent type. Mengden slokningsmiddel fastsettes i hvert enkelt tilfelle avhengig av maskinrommets størrelse.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "aaa524b9-efda-4bca-b118-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/15/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/15/ledd/3",
                    "content": "(3) Slokningsanlegget skal bare kunne utløses utenfor maskinrommet. Utløserutstyret skal være tydelig merket, utstyrt med bruksinstruksjon og beskyttet mot utilsiktet utløsning.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "3222599a-82d4-42fc-b119-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/15/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/15/ledd/4",
                    "content": "(4) Fartøy med største lengde mindre enn 10,67 meter bygget etter 1. januar 1992 kan alternativt være utstyrt med brannslokningsanlegg i samsvar med retningslinjer fastsatt i kapittel Y14 i Nordisk Båt Standard (1990).",
                    "subparts": [],
                    "sentences": [],
                    "uid": "f95f1eec-f545-421a-b11a-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/15/part/4"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/15/ledd/5",
                    "content": "(5) CO -beholdere og andre beholdere for slokningsgass som kan medføre fare ved lekkasje, med tilhørende utløsningsanordning, skal være plassert i eget rom/skap utenfor maskinrom. Det skal være gasstette skott til tilstøtende rom og inngang fra åpent dekk. Et alarmsignal skal automatisk tre i funksjon når CO -rommet/skapet åpnes. Man må forsikre seg om at ingen personer er tilstede i motorrom før CO -gassen utløses. Rommet/skapet for utløsning av CO skal ha ventilasjon til fri luft.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "18ae10ec-cc12-4bb4-b11b-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/15/part/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/15",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p15",
            "title": "Brannslokningsanlegg i maskinrom",
            "uid": "e39c5f0c-0383-4ed5-9e99-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/15"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/16/ledd/1",
                    "content": "(1) Fartøy mellom største lengde 10,67 og 15 meter, uavhengig av byggeår, skal ha brannoppdagelsesanlegg og brannalarmanlegg for maskinrom og røykvarslingsanlegg for lugarene i samsvar med vedlegg 5.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "e8cb7b5b-7bbf-44c4-b11c-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/16/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/16/ledd/2",
                    "content": "(2) Fartøy mellom største lengde under 10,67 meter skal være utstyrt med minst én røyk- og én varmedetektor i maskinrom, minst én røykdetektor i oppholdsrom, og minst én røykdetektor i styrehus.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "86f8abf5-16cc-44d8-b11d-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/16/part/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/16",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p16",
            "title": "Brannoppdagelses- og alarmanlegg",
            "uid": "1934142b-2fef-467c-9e9a-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/16"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/17/ledd/1",
                    "content": "(1) Alle fartøy, uavhengig av byggeår, skal ha minst to godkjente brannslokningsapparat på minst 6 kg. Minst ett godkjent brannslokningsapparat skal plasseres på egnet sted utenfor maskinrommet og minst ett godkjent brannslokningsapparat skal plasseres i innredningen. For fartøy uten styrehus eller innredning er det tilstrekkelig med ett brannslokningsapparat.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "81b5a4f0-8d47-4f6c-b11e-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/17/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/17/ledd/2",
                    "content": "(2) Brannslukningsapparater med slokningsmiddel under trykk skal ikke plasseres i lugarer. Brannslukningsapparater som kan bli utsatt for frost, skal være frostsikre.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "266f0a0d-4685-4032-b11f-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/17/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/17/ledd/3",
                    "content": "(3) Det skal være minst en reserveladning til hvert apparat. Apparatenes antall skal økes med 50prosent om det er av en type som ikke kan lades opp ombord.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "73e7dde2-cb06-48f3-b120-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/17/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/17/ledd/4",
                    "content": "(4) Brannslokningsapparater skal årlig kontrolleres i henhold til NS3910. Datoen for kontrollen skal påføres apparatet.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "bb09533b-0b29-4ed5-b121-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/17/part/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/17",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p17",
            "title": "Brannslokningsapparater",
            "uid": "7b66ca37-bdaf-4c4c-9e9b-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/17"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/18/ledd/1",
                    "content": "Fartøyets maskinrom skal ha minst to ventilatorer som kan stenges fra dekk. Lugarer skal ha minst to ventilatorer med diameter på minst 8 cm.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ac65ded3-f4da-4d61-b122-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/18/part/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/18",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p18",
            "title": "Ventilasjonssystemer",
            "uid": "547a0e3d-b988-4bd5-9e9c-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/18"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/19/ledd/1",
                    "content": "Materialer, lakk, maling mv. fremstilt på nitrocellulosebasis eller annen lett antennelig basis, tillates ikke benyttet noe sted om bord.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "65c51e8f-66d8-46c7-b123-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/19/part/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/19",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p19",
            "title": "Materialer m.m.",
            "uid": "ec6eb09e-1f4c-498c-9e9d-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/19"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/20/ledd/1",
                    "content": "(1) Brennolje skal oppbevares på tanker av stål eller likeverdig materiale.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "ab3a2138-72fb-446d-b124-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/20/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/20/ledd/2",
                    "content": "(2) Avløp fra brennoljetank skal være utstyrt med avstengningsventil eller kran direkte på tanken, stengbar fra dekk. Prøvekraner og kraner for oljestandsglass skal være av selvlukkende type. Tanken skal være utstyrt med spillbrett. På dekket fartøy med maskinrom skal påfyllingsrør og avluftningsrør være ført gjennom dekk ut til fri luft.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "37a5a0da-565f-44c7-b125-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/20/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/20/ledd/3",
                    "content": "(3) Dersom ekstra brennstoff eller andre brennbare væsker ikke kan anbringes på åpent dekk, skal de oppbevares i egnede beholdere i et godt ventilert rom/skap med dør kun til åpent dekk og uten elektriske eller andre tennkilder.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "23ee0806-3849-409a-b126-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/20/part/3"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/20",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p20",
            "title": "Oppbevaring av brennolje og andre brennbare væsker",
            "uid": "fda1e0c9-4ff2-4aea-9e9e-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/20"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/21/ledd/1",
                    "content": "(1) Sjøvannsledninger, lenseledninger, rør som fører olje, og andre rørledninger nødvendige for skipets sikkerhet, skal være av ubrennbart materiale. Korte fleksible rørforbindelser av et materiale som ikke lett gjøres ubrukbart av varme kan godtas på\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/21/ledd/1/liste/a",
                            "content": "sjøvannsledninger,",
                            "uid": "cabb5e01-90b9-420b-f397-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/21/part/1/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/21/ledd/1/liste/b",
                            "content": "korte fleksible rørforbindelser på rør som fører olje når det er nødvendig på grunn av vibrasjoner.",
                            "uid": "033f17f1-f1c8-4b2f-f398-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/21/part/1/subpart/2"
                        }
                    ],
                    "sentences": [],
                    "uid": "d7c25624-5b61-4b0a-b127-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/21/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/21/ledd/2",
                    "content": "(2) Spygatt, sanitære avløp eller andre avløp nær vannlinjen hvor svikt i materialet som følge av brann kan forårsake innstrømning av vann skal være av oljebestandige og armerte materialer som ikke lett skades av varme.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "e917654c-a241-41ae-b128-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/21/part/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/21",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p21",
            "title": "Materialer i rørledninger mv.",
            "uid": "b5505dc8-49b0-4f94-9e9f-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/21"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/22/ledd/1",
                    "content": "(1) Fartøy bygget etter 1. januar 1992 skal oppfylle kravene i Nordisk Båt Standard (1990)\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/22/ledd/1/liste/a",
                            "content": "Y6 om lensesystemer,",
                            "uid": "9d1788bd-e7ea-4948-f399-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/1/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/22/ledd/1/liste/b",
                            "content": "Y9 om brennoljesystemer,",
                            "uid": "d41c93bc-b93d-4bf7-f39a-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/1/subpart/2"
                        },
                        {
                            "number": "c)",
                            "referenceId": "/kapittel/3/paragraf/22/ledd/1/liste/c",
                            "content": "Y8 om motorinstallasjon, og",
                            "uid": "b4f992cf-4c12-4115-f39b-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/1/subpart/3"
                        },
                        {
                            "number": "d)",
                            "referenceId": "/kapittel/3/paragraf/22/ledd/1/liste/d",
                            "content": "Y7 og Y10 om ror og propellarrangement.",
                            "uid": "b3458e90-24e9-4973-f39c-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/1/subpart/4"
                        }
                    ],
                    "sentences": [],
                    "uid": "15e9f2a4-7ff2-4181-b129-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/22/ledd/2",
                    "content": "(2) Fartøy bygget før 1. januar 1992 skal så langt det er mulig oppfylle kravene i Nordisk Båt Standard (1990) kapittel Y6 om lensesystemer.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "24080ef3-2bee-4742-b12a-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/22/part/2"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/22",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p22",
            "title": "Maskineri og rørsystemer",
            "uid": "c6b74a79-0bde-4add-9ea0-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/22"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/23/ledd/1",
                    "content": "(1) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 1992 med elektriske anlegg med spenning inntil 50 V skal oppfylle kravene i Nordisk Båt Standard (1990) Y11 om elektriske anlegg.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "73ff7a5f-68fd-47cc-b12b-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/23/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/23/ledd/2",
                    "content": "(2) Fartøy med største lengde 6 til 15 meter bygget før 1. januar 1992 med elektriske anlegg med spenning inntil 50 V skal være i henhold til regelverket som gjaldt ved byggetidspunktet.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "85b2273b-8073-43b9-b12c-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/23/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/23/ledd/3",
                    "content": "(3) Fartøy med største lengde 6 til 15 meter bygget mellom 1. januar 1992 og 2002 med elektrisk anlegg med spenning over 50 V skal tilfredsstille kravene fastsatt i den nå opphevede forskrift 1. mars 1990 nr. 125 for elektriske anlegg - Maritime installasjoner.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "3d6b9bd9-dd60-4957-b12d-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/23/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/23/ledd/4",
                    "content": "(4) Fartøy med største lengde 6 til 15 meter bygget etter 1. januar 2002 med elektrisk anlegg med spenning over 50 V skal tilfredsstille kravene fastsatt i forskrift 4. desember 2001 nr. 1450 om maritime elektriske anlegg utgitt av Direktoratet for samfunnssikkerhet og beredskap.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "704f55ce-f0c8-410d-b12e-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/23/part/4"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/23",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p23",
            "title": "Elektriske anlegg",
            "uid": "33626bc9-9d45-4578-9ea1-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/23"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/24/ledd/1",
                    "content": "(1) Fartøy med største lengde 10,67 til 15 meter bygget etter 1. januar 1992 skal ha nødkraft som gir tilfredsstillende belysning i minst 2 timer tilstrekkelig til\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/1/liste/a",
                            "content": "belysning i område med redningsmidler og utsetningsmateriell,",
                            "uid": "ad4f027e-b045-4384-f39d-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/1/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/1/liste/b",
                            "content": "belysning i ganger, trapper og utganger til redningsmidlene og av sjøen hvor redningsmidlene settes ut,",
                            "uid": "d9de8863-8785-4241-f39e-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/1/subpart/2"
                        },
                        {
                            "number": "c)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/1/liste/c",
                            "content": "nødvendige navigasjonslys, og",
                            "uid": "2d7e5687-2045-4a89-f39f-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/1/subpart/3"
                        },
                        {
                            "number": "d)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/1/liste/d",
                            "content": "pålagt alarmsystem.",
                            "uid": "2e617601-d488-4d40-f3a0-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/1/subpart/4"
                        }
                    ],
                    "sentences": [],
                    "uid": "a16b390b-d78e-4f7d-b12f-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/1"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/24/ledd/2",
                    "content": "(2) Nødkraften skal komme fra en av følgende to kilder\n",
                    "subparts": [
                        {
                            "number": "a)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/2/liste/a",
                            "content": "akkumulatorbatteri som kan tåle belastningen etter første ledd uten oppladning eller for stort spenningsfall og som automatisk kobles til nødstrømtavla når hovedkraften svikter, eller",
                            "uid": "76d7014a-a371-4786-f3a1-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/2/subpart/1"
                        },
                        {
                            "number": "b)",
                            "referenceId": "/kapittel/3/paragraf/24/ledd/2/liste/b",
                            "content": "en generator drevet av en passende kraftmaskin med selvstendig brenselforsyning, en hensiktsmessig startanordning og drevet med brensel med flammepunkt på 43 °C (110 °F) eller mer.",
                            "uid": "a0793e2f-2d64-4ba6-f3a2-08d8c4369723",
                            "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/2/subpart/2"
                        }
                    ],
                    "sentences": [],
                    "uid": "fd705bed-93be-4f57-b130-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/2"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/24/ledd/3",
                    "content": "(3) Hele installasjonen skal fungere selv om skipet har 22,5 grader slagside eller skipets trim er 10 grader.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "6a5ba912-4de4-483b-b131-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/3"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/24/ledd/4",
                    "content": "(4) Tavler for hovedkraft og nødkraft skal være tydelig merket og så langt som mulig plasseres slik at de ikke kan utsettes for vann eller ild. Nødkraftkilden skal om mulig plasseres over dekk. Plasseres nødkraftkilden i maskinrommet, skal den plasseres over dørk og på et godt ventilert sted.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "be4f0dd2-72ae-4eee-b132-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/4"
                },
                {
                    "referenceId": "/kapittel/3/paragraf/24/ledd/5",
                    "content": "(5) Nødkraften og nødbelysningen i fartøy bygget før 1. januar 1992 skal være i henhold til regelverket som gjaldt ved byggetidspunktet.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "4227b17f-15fc-4a00-b133-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/24/part/5"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/24",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p24",
            "title": "Nødkraftkilde og nødlys, mv.",
            "uid": "538af352-b52c-451f-9ea2-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/24"
        },
        {
            "parts": [
                {
                    "referenceId": "/kapittel/3/paragraf/25/ledd/1",
                    "content": "Fartøy som er bygget etter Nordisk Båt Standard (1990) anses å tilfredsstille bestemmelsene i § 18, § 19 og § 20.",
                    "subparts": [],
                    "sentences": [],
                    "uid": "6127d3e0-e4ce-4ad2-b134-08d8c4369707",
                    "url": "/regulation/2013/11/22/1404/chapter/3/section/25/part/1"
                }
            ],
            "tables": [],
            "referenceId": "/kapittel/3/paragraf/25",
            "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3/p25",
            "title": "Likeverdige løsninger",
            "uid": "96858768-9b75-4b79-9ea3-08d8c4369704",
            "url": "/regulation/2013/11/22/1404/chapter/3/section/25"
        }
    ],
    "referenceId": "/kapittel/3",
    "referenceUrl": "https://lovdata.no/forskrift/2013-11-22-1404/k3",
    "title": "Brann, maskineri og elektro",
    "uid": "881f7a98-8dc1-4297-9693-08d8c43696f5",
    "url": "/regulation/2013/11/22/1404/chapter/3"
}

In [6]:
#
# Chapter info
#

print(text_service_chapter_input['referenceId']) # /kapittel/3
print(text_service_chapter_input['referenceUrl']) # https://lovdata.no/forskrift/2013-11-22-1404/k3
print(text_service_chapter_input['title']) # Brann, maskineri og elektro
print(text_service_chapter_input['uid']) # da8ef9e9-ce56-49d6-94d3-08d8c109f9af
print(text_service_chapter_input['url']) # http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/25

number_of_sections_in_chapter = len(text_service_chapter_input['sections'])
print("number_of_sections_in_chapter: " + str(number_of_sections_in_chapter))
print("-------")

for section_index_number, section_item in enumerate(text_service_chapter_input['sections']):
    #section_number = section_index_number + 1
    print(section_item['referenceId'] + " - " + section_item['title'])

    number_of_subsections = len(section_item['parts'])
    print("number_of_parts: " + str(number_of_subsections))

    for subsection_index_number, subsection_item in enumerate(section_item['parts']):
        #"uid": "ca473934-8e31-4d4b-a821-08d8c109f9c0",
        #"url": "http://sdir-d-aks-core.norwayeast.cloudapp.azure.com/v1/regulations/2013/11/22/1404/chapters/3/sections/10/subsections/9"

        number_of_sentences = len(subsection_item['subparts'])
        print("number_of_subparts: " + str(number_of_sentences))

/kapittel/3
https://lovdata.no/forskrift/2013-11-22-1404/k3
Brann, maskineri og elektro
881f7a98-8dc1-4297-9693-08d8c43696f5
/regulation/2013/11/22/1404/chapter/3
number_of_sections_in_chapter: 19
-------
/kapittel/3/paragraf/7 - Kapittelets anvendelsesområde
number_of_parts: 3
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
/kapittel/3/paragraf/8 - Brannsikring
number_of_parts: 3
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
/kapittel/3/paragraf/9 - Oljefyrt ovn
number_of_parts: 7
number_of_subparts: 3
number_of_subparts: 2
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
/kapittel/3/paragraf/10 - Installasjon mv. av oljefyrt ovn
number_of_parts: 10
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
number_of_subparts: 0
/kapittel/3/paragraf/1

# Functions run from .py file

In [7]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import get_data_from_text_service_item_url

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

for section_item in text_service_chapter_input['sections']:    
    for subsection_item in section_item['parts']:

        data = get_data_from_text_service_item_url(subsection_item['url'])

        if all (k in data for k in ("section_number","part_number","sub_part_number")):
            print("part - section: " + str(data['section_number']) + " part: " + str(data['part_number']) + " sub: " + data['sub_part_number'])
        elif all (k in data for k in ("section_number","part_number")):
            print("part - section: " + str(data['section_number']) + " part: " + str(data['part_number']))
        elif "section_number" in data:
            print("part - section: " + str(data['section_number']))
        else:
            print("part")

        for sentence_item in subsection_item['subparts']:

            data = get_data_from_text_service_item_url(sentence_item['url'])

            if all (k in data for k in ("section_number","part_number","sub_part_number")):
                print("subpart - section: " + str(data['section_number']) + " part: " + str(data['part_number']) + " sub: " + data['sub_part_number'])
            elif all (k in data for k in ("section_number","part_number")):
                print("subpart - section: " + str(data['section_number']) + " part: " + str(data['part_number']))
            elif "section_number" in data:
                print("subpart - section: " + str(data['section_number']))
            else:
                print("subpart")

part - section: 7 part: 1
part - section: 7 part: 2
part - section: 7 part: 3
part - section: 8 part: 1
part - section: 8 part: 2
part - section: 8 part: 3
part - section: 9 part: 1
subpart - section: 9 part: 1 sub: 1
subpart - section: 9 part: 1 sub: 2
subpart - section: 9 part: 1 sub: 3
part - section: 9 part: 2
subpart - section: 9 part: 2 sub: 1
subpart - section: 9 part: 2 sub: 2
part - section: 9 part: 3
part - section: 9 part: 4
part - section: 9 part: 5
part - section: 9 part: 6
part - section: 9 part: 7
part - section: 10 part: 1
part - section: 10 part: 2
part - section: 10 part: 3
part - section: 10 part: 4
part - section: 10 part: 5
part - section: 10 part: 6
part - section: 10 part: 7
part - section: 10 part: 8
part - section: 10 part: 9
part - section: 10 part: 10
part - section: 11 part: 1
part - section: 11 part: 2
part - section: 12 part: 1
part - section: 12 part: 2
part - section: 12 part: 3
part - section: 12 part: 4
part - section: 12 part: 5
part - section: 13 par

In [8]:
#
# Something that must be done 
# to make the notebook reload a function
# from a costum .py file.
#

%reload_ext autoreload
%autoreload 2

import importlib

import transform_text_service_input_to_spacy_format

from transform_text_service_input_to_spacy_format import transform_chapter_from_text_service_to_spacy_format

importlib.reload(transform_text_service_input_to_spacy_format)

################
################

transform_chapter_from_text_service_to_spacy_format(text_service_chapter_input)

[{'text': '(1) Dette kapittelet gjelder for fartøy bygget etter 2. januar 1988.',
  'ents': [],
  'title': '/regulation/2013/11/22/1404/chapter/3/section/7/part/1'},
 {'text': '(2) Fartøy som er bygget før 2. januar 1988 skal være bygget og utrustet etter reglene som gjaldt da fartøyet ble bygget, med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.',
  'ents': [],
  'title': '/regulation/2013/11/22/1404/chapter/3/section/7/part/2'},
 {'text': '(3) Når et fartøy forandres eller ombygges i vesentlig omfang, skal fartøyet oppfylle kravene til nytt fartøy.',
  'ents': [],
  'title': '/regulation/2013/11/22/1404/chapter/3/section/7/part/3'},
 {'text': '(1) Fartøy bygget etter 1. januar 1992 skal brannsikres konstruksjonsmessig i henhold til kapittel Y14 i Nordisk Båt Standard (1990), med mindre annet fremgår av de enkelte bestemmelsene i dette kapittelet.',
  'ents': [],
  'title': '/regulation/2013/11/22/1404/chapter/3/section/8/part/1'},
 {'text': '(2) Fartøy som er b